In [ ]:
import email.parser

%load_ext autoreload
%autoreload 2

import os
from email import policy
from email.parser import BytesParser

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl

from src.spam_classifier.constants import PROJECT_ROOT

DATA_DIR = PROJECT_ROOT / "data"

In [ ]:
def parse_email(filepath):
    with open(filepath, "rb") as f:
        msg = BytesParser(policy=policy.default).parse(f)

    subject = msg["subject"]
    sender = msg["from"]

    body = ""
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == "text/plain" and not part.get_content_disposition():
                body = part.get_content()
                break
    else:
        body = msg.get_content()

    return {
        "subject": subject,
        "sender": sender,
        "body": body,
    }